In [23]:
import os

project_path = r"C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project"

os.chdir(project_path)
print(f"Moved to: {os.getcwd()}")


Moved to: C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project


In [24]:
%pwd

'C:\\Users\\user\\anaconda3\\envs\\Chicken-disease-Classification-Project'

In [25]:
import tensorflow as tf

In [26]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [27]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class EvaluationConfig:
    trained_model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: List[int]
    params_batch_size: int


In [28]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml, create_directories, save_json

In [30]:
class ConfigurationManager:
    def __init__(self, config_filepath: str = CONFIG_FILE_PATH, params_filepath: str = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_validation_config(self) -> EvaluationConfig:
        model_params = self.params.get("MODEL_PARAMS", {})
    
        image_size = model_params.get("IMAGE_SIZE")
        batch_size = model_params.get("BATCH_SIZE")
    
        if image_size is None or batch_size is None:
            raise KeyError("IMAGE_SIZE or BATCH_SIZE not found in MODEL_PARAMS section of params.yaml")
    
        eval_config = EvaluationConfig(
            trained_model_path=os.path.join("artifacts", "training", "model.keras"),
            training_data=os.path.join("artifacts", "data_ingestion", "Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=image_size,
            params_batch_size=batch_size
        )
        return eval_config


In [31]:
from urllib.parse import urlparse

In [ ]:
from urllib.parse import urlparse
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _load_model(self):
        self.model = tf.keras.models.load_model(self.config.trained_model_path)

    def _load_validation_data(self):
        datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1.0 / 255,
            validation_split=0.2  # same as training
        )

        self.val_generator = datagen.flow_from_directory(
            directory=self.config.training_data,
            target_size=self.config.params_image_size[:2],
            batch_size=self.config.params_batch_size,
            class_mode="categorical",
            subset="validation",
            shuffle=False  # important for label alignment
        )

    def _evaluate_model(self):
        self._load_model()
        self._load_validation_data()

        self.score = self.model.evaluate(self.val_generator)
        self.y_pred = self.model.predict(self.val_generator)
        self.y_pred_classes = np.argmax(self.y_pred, axis=1)
        self.y_true = self.val_generator.classes

        self.classification_report = classification_report(
            self.y_true, self.y_pred_classes, output_dict=True
        )
        self.conf_matrix = confusion_matrix(self.y_true, self.y_pred_classes)

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1],
            "classification_report": self.classification_report,
            "confusion_matrix": self.conf_matrix.tolist()
        }

        save_path = Path("artifacts/evaluation/scores.json")
        save_path.parent.mkdir(parents=True, exist_ok=True)  # Create directory if missing

        save_json(path=save_path, data=scores)

    def evaluate(self):
        self._evaluate_model()
        self.save_score()


In [38]:
from cnnclassifier import logger  # your custom logger

try:
    logger.info("Initializing ConfigurationManager...")
    config = ConfigurationManager()
    logger.info("ConfigurationManager initialized.")

    logger.info("Fetching evaluation configuration...")
    val_config = config.get_validation_config()  # call the method
    logger.info("Evaluation configuration fetched.")

    logger.info("Initializing Evaluation class...")
    evaluation = Evaluation(val_config)
    logger.info("Evaluation class initialized.")

    logger.info("Starting model evaluation...")
    evaluation.evaluate()  # call your evaluation method
    logger.info("Model evaluation completed successfully.")

    logger.info("Saving evaluation scores...")
    evaluation.save_score()  # call method to save scores
    logger.info("Evaluation scores saved successfully.")

except Exception as e:
    logger.exception("An error occurred during the evaluation pipeline execution.")
    raise e


[2025-06-05 16:14:39,754: INFO: 2793454304: Initializing ConfigurationManager...]
[2025-06-05 16:14:39,762: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-06-05 16:14:39,773: INFO: common: YAML file: params.yaml loaded successfully]
[2025-06-05 16:14:39,776: INFO: common: Created directory at: artifacts]
[2025-06-05 16:14:39,778: INFO: 2793454304: ConfigurationManager initialized.]
[2025-06-05 16:14:39,778: INFO: 2793454304: Fetching evaluation configuration...]
[2025-06-05 16:14:39,780: INFO: 2793454304: Evaluation configuration fetched.]
[2025-06-05 16:14:39,783: INFO: 2793454304: Initializing Evaluation class...]
[2025-06-05 16:14:39,783: INFO: 2793454304: Evaluation class initialized.]
[2025-06-05 16:14:39,783: INFO: 2793454304: Starting model evaluation...]
Found 78 images belonging to 2 classes.


c:\Users\user\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.9672 - loss: 0.1409
5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step
[2025-06-05 16:15:25,187: INFO: common: JSON file saved at: artifacts\evaluation\scores.json]
[2025-06-05 16:15:25,194: INFO: 2793454304: Model evaluation completed successfully.]
[2025-06-05 16:15:25,197: INFO: 2793454304: Saving evaluation scores...]
[2025-06-05 16:15:25,201: INFO: common: JSON file saved at: artifacts\evaluation\scores.json]
[2025-06-05 16:15:25,204: INFO: 2793454304: Evaluation scores saved successfully.]
